In [ ]:
import shutil
from sentence_transformers import SentenceTransformer
import chromadb
import json

In [ ]:
try:
    shutil.rmtree(r"C:\last_project\make_domain_selector\syntax_info")
except:
    pass

In [ ]:
syntax_info = []

with open(r"C:\last_project\make_domain_selector\mimic_oracle_syntax.json", "r", encoding="utf-8") as f:
    oracle_syntax_info = json.load(f)

In [45]:
for key in oracle_syntax_info.keys():
    syntax_for_rag = f'[{key}]\n'
    syntax_for_rag += ('\n'.join([f'{str(i+1)}.{oracle_syntax_info[key][i]}' for i in range(len(oracle_syntax_info[key]))]))
    syntax_info += ["passage: " + syntax_for_rag]

In [46]:
embedding_model = SentenceTransformer("intfloat/e5-small-v2")

def embed_passages(texts):
    texts = [f"passage: {t}" for t in texts]
    return embedding_model.encode(texts, batch_size=64, show_progress_bar=True).tolist()

client = chromadb.PersistentClient(
    path=r"C:\last_project\make_domain_selector\\syntax_info"
)

# 기존 컬렉션 삭제
try:
    client.delete_collection("syntax_info")
except:
    pass

collection = client.create_collection(name="syntax_info")

embeddings = embed_passages(syntax_info)

collection.add(
    documents=syntax_info,
    embeddings=embed_passages(syntax_info),
    ids= [f'table_{i}' for i in range(len(syntax_info))])

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.94it/s]


In [47]:
def embed_query(text):
    """검색 쿼리 임베딩용"""
    if isinstance(text, str):
        text = [text]
    texts = [f"query: {t}" for t in text]
    return embedding_model.encode(texts, show_progress_bar=True).tolist()

In [49]:
collection = client.get_collection("syntax_info")

# 검색 시 수정
results = collection.query(
    query_embeddings=embed_query("Find the average length of stay for each ICU ward."),  # "query:" prefix 사용
    n_results=5
)

print(results["documents"][0][0])
print(results["metadatas"][0][0])

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]

passage: [ICU_LOS_RULES]
1.For ICU stay duration analysis, ALWAYS use icustays.los.
2.NEVER calculate ICU duration using procedureevents starttime and endtime.
3.LOS is a direct column name in icustays.
4.Do not search for itemid mapping for LOS.
None
